In [3]:
!pip install ibm_watson bs4

     |████████████████████████████████| 378kB 8.6MB/s eta 0:00:01
     |████████████████████████████████| 204kB 8.8MB/s eta 0:00:01
     |████████████████████████████████| 122kB 32.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/71/9a/0a/9b3ca8eca69bc5362eb04709a750b30055a9d27818fd0c9494
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/a0/b0/b2/4f80b9456b87abedbc0bf2d52235414c3467d8889be38dd472
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson bs4 ibm-cloud-sdk-core


In [4]:
import os
from glob import glob
from bs4 import BeautifulSoup
import IPython
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import SpeechToTextV1
from ibm_watson import AssistantV2
from ibm_watson import TextToSpeechV1

In [6]:
recognition_service = SpeechToTextV1(IAMAuthenticator('mtktish5Xd2nqC_7p2tdF6iNnJaMhRx4Y4ijc7Ox2BtS'))
recognition_service.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/b7f6325f-5645-42bb-b2c5-31003cf6fee4/v2/assistants/c4305fb8-e2ff-4142-806d-39c8b7805071/sessions')
SPEECH_EXTENSION = "*.webm"#"*.webm"
SPEECH_AUDIOTYPE = "audio/webm"#"audio/webm"

def recognize_audio():
    while len(glob(SPEECH_EXTENSION)) == 0:
        pass
    filename = glob(SPEECH_EXTENSION)[0]
    audio_file = open(filename, "rb")
    os.popen("rm " + filename)
    result = recognition_service.recognize(audio=audio_file, content_type=SPEECH_AUDIOTYPE).get_result()
    return result["results"][0]["alternatives"][0]["transcript"]

In [ ]:
assistant = AssistantV2(version='2020-04-01', authenticator=IAMAuthenticator('mtktish5Xd2nqC_7p2tdF6iNnJaMhRx4Y4ijc7Ox2BtS'))
assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/b7f6325f-5645-42bb-b2c5-31003cf6fee4/v2/assistants/c4305fb8-e2ff-4142-806d-39c8b7805071/sessions')#https://api.us-south.assistant.watson.cloud.ibm.com/instances/b7f6325f-5645-42bb-b2c5-31003cf6fee4/v2/assistants/c4305fb8-e2ff-4142-806d-39c8b7805071/sessions
ASSISTANT_ID = "c4305fb8-e2ff-4142-806d-39c8b7805071"
session_id = assistant.create_session(assistant_id=ASSISTANT_ID).get_result()["session_id"]

def message_assistant(text):
    response = assistant.message(assistant_id=ASSISTANT_ID,
                                 session_id=session_id,
                                 input={'message_type': 'text', 'text': text}).get_result()
    return BeautifulSoup(response["output"]["generic"][0]["text"]).get_text()

In [ ]:
synthesis_service = TextToSpeechV1(IAMAuthenticator('mtktish5Xd2nqC_7p2tdF6iNnJaMhRx4Y4ijc7Ox2BtS'))
synthesis_service.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/b7f6325f-5645-42bb-b2c5-31003cf6fee4/v2/assistants/c4305fb8-e2ff-4142-806d-39c8b7805071/sessions')

def speak_text(text):
    with open('temp.wav', 'wb') as audio_file:
        response = synthesis_service.synthesize(text, accept='audio/wav', voice="en-US_AllisonV3Voice").get_result()
        audio_file.write(response.content)
    return IPython.display.Audio("temp.wav", autoplay=True)

In [ ]:
speak_text(message_assistant(recognize_audio()))